In [2]:
!pip install python-dotenv



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from weather_api import fetch_weather_forecast
from datetime import date

# Указываем координаты (например, для Никосии, Кипр)
latitude = 35.1856
longitude = 33.3823
target_date = date.today()

forecast = fetch_weather_forecast(latitude, longitude, target_date)

if forecast:
    import pandas as pd
    pd.DataFrame(forecast).head()
else:
    print("Прогноз не получен.")


In [4]:
from datetime import date, timedelta
import logging
import os
from typing import List, Optional

import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv

logger = logging.getLogger(__name__)
load_dotenv()

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY", "0d8ab490dd584b518ee91719252111")
FORECAST_API_URL = os.getenv("WEATHER_API_URL", "https://api.weatherapi.com/v1/forecast.json")
HISTORY_API_URL = os.getenv("WEATHER_HISTORY_API_URL", "https://api.weatherapi.com/v1/history.json")

def _resample_to_quarter_hour(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["time"] = pd.to_datetime(df["time"])
    df = df.set_index("time").sort_index()
    numeric_cols = [c for c in df.columns if c != "time"]
    df = df.resample("15min").interpolate()
    df = df[numeric_cols].reset_index()
    df["time"] = df["time"].dt.strftime("%Y-%m-%d %H:%M")
    return df

def _add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    dt_idx = pd.to_datetime(df["time"])
    df["hour_local"] = dt_idx.dt.hour + dt_idx.dt.minute / 60.0
    df["dayofyear"] = dt_idx.dt.dayofyear

    df["hour_sin"] = np.sin(df["hour_local"] * 2 * np.pi / 24)
    df["hour_cos"] = np.cos(df["hour_local"] * 2 * np.pi / 24)
    df["doy_sin"] = np.sin(df["dayofyear"] * 2 * np.pi / 365)
    df["doy_cos"] = np.cos(df["dayofyear"] * 2 * np.pi / 365)
    return df

def fetch_weather_forecast(latitude: float, longitude: float, target_date: date) -> Optional[List[dict]]:
    if not WEATHER_API_KEY:
        logger.error("WEATHER_API_KEY is not set")
        return None

    api_url = FORECAST_API_URL if target_date >= date.today() else HISTORY_API_URL
    params = {
        "key": WEATHER_API_KEY,
        "q": f"{latitude},{longitude}",
        "dt": target_date.strftime("%Y-%m-%d"),
        "aqi": "no",
        "alerts": "no",
    }

    try:
        response = requests.get(api_url, params=params, timeout=15)
        response.raise_for_status()
    except requests.RequestException as exc:
        logger.error("Weather API request failed: %s", exc)
        return None

    payload = response.json()

    # Проверка forecastday
    forecast_days = payload.get("forecast", {}).get("forecastday", [])
    if forecast_days:
        hours = forecast_days[0].get("hour", [])
    else:
        # fallback на current
        current = payload.get("current", {})
        if not current:
            logger.error("No forecast or current weather data found")
            return None
        hours = [{
            "time": payload["location"]["localtime"],
            "temp_c": current.get("temp_c"),
            "cloud": current.get("cloud"),
        }]

    if not hours:
        logger.error("No hourly or current data found in Weather API response")
        return None

    df = pd.DataFrame(hours)
    if df.empty:
        return None

    df = _resample_to_quarter_hour(df)
    df = _add_time_features(df)
    return df.to_dict(orient="records")


In [5]:
from datetime import date

latitude = 35.1856
longitude = 33.3823
target_date = date.today()

forecast = fetch_weather_forecast(latitude, longitude, target_date)

if forecast:
    import pandas as pd
    display(pd.DataFrame(forecast).head())
else:
    print("Прогноз не получен.")


C:\Users\Dmitriy\AppData\Local\Temp\ipykernel_12080\2773471720.py:23: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.resample("15min").interpolate()


,time,time_epoch,temp_c,temp_f,is_day,condition,wind_mph,wind_kph,wind_degree,wind_dir,...,short_rad,diff_rad,dni,gti,hour_local,dayofyear,hour_sin,hour_cos,doy_sin,doy_cos
0,2025-11-21 00:00,1.763676e+09,17.800,64.000,0.0,"{'text': 'Clear ', 'icon': '//cdn.weatherapi.c...",4.0,6.500,45.0,NE,...,0.0,0.0,0.0,0.0,0.00,325,0.000000,1.000000,-0.635432,0.772157
1,2025-11-21 00:15,1.763677e+09,17.725,63.875,0.0,NaN,3.9,6.325,39.0,NaN,...,0.0,0.0,0.0,0.0,0.25,325,0.065403,0.997859,-0.635432,0.772157
2,2025-11-21 00:30,1.763678e+09,17.650,63.750,0.0,NaN,3.8,6.150,33.0,NaN,...,0.0,0.0,0.0,0.0,0.50,325,0.130526,0.991445,-0.635432,0.772157
3,2025-11-21 00:45,1.763679e+09,17.575,63.625,0.0,NaN,3.7,5.975,27.0,NaN,...,0.0,0.0,0.0,0.0,0.75,325,0.195090,0.980785,-0.635432,0.772157
4,2025-11-21 01:00,1.763680e+09,17.500,63.500,0.0,"{'text': 'Clear ', 'icon': '//cdn.weatherapi.c...",3.6,5.800,21.0,NNE,...,0.0,0.0,0.0,0.0,1.00,325,0.258819,0.965926,-0.635432,0.772157


In [6]:
from datetime import date
import pandas as pd

# Координаты и дата
latitude = 35.1856
longitude = 33.3823
target_date = date.today()

# Получаем прогноз
forecast = fetch_weather_forecast(latitude, longitude, target_date)

if forecast:
    # Извлекаем почасовые данные
    hourly_data = forecast["forecast"]["forecastday"][0]["hour"]
    
    # Преобразуем в DataFrame
    df = pd.DataFrame(hourly_data)

    # Оставляем только нужные поля
    df_selected = df[[
        "time", "temp_c", "is_day", "condition", "wind_kph", "humidity",
        "cloud", "feelslike_c", "uv", "short_rad", "diff_rad", "dni", "gti"
    ]]

    # Распечатываем первые строки для проверки
    display(df_selected.head())
else:
    print("Прогноз не получен.")


C:\Users\Dmitriy\AppData\Local\Temp\ipykernel_12080\2773471720.py:23: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.resample("15min").interpolate()


TypeError: list indices must be integers or slices, not str